This Notebook Explore small data about Combined Cycle Power Plant Data Set from website UCI.

Link for data set : https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant

This data set consist 4 feature and based on this 4 feature net hourly electrical energy output (EP) is calculated/estimated.

We are going to use different regression models, evalute their performance and compare their performance with each other.

In [9]:
import numpy as np
import pyexcel as p
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn import linear_model as lm
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor

In [2]:
#reading excel data in pandas dataframe
fname = 'data.xlsx'
dff = pd.read_excel(fname)
dff.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [3]:
#dividing data into input and output dataframes. Input has 4 features
#(Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V))
#Outout is  net hourly electrical energy output (PE) 
y = dff[dff.columns[4:]]
x = dff[dff.columns[0:4]]
x.head()

,AT,V,AP,RH
0,14.96,41.76,1024.07,73.17
1,25.18,62.96,1020.04,59.08
2,5.11,39.40,1012.16,92.14
3,20.86,57.32,1010.24,76.64
4,10.82,37.50,1009.23,96.62


In [4]:
#in this section we will perform feature scanling. 
#range of 4 features are as follows
#Temperature (T) in the range 1.81°C and 37.11°C,
#Ambient Pressure (AP) in the range 992.89-1033.30 milibar,
#Relative Humidity (RH) in the range 25.56% to 100.16%
#Exhaust Vacuum (V) in teh range 25.36-81.56 cm Hg
for col in x:
    min_ = x[col].min()
    max_ = x[col].max()
    exp = lambda x : (x-min_)/(max_-min_)
    x[col] = x[col].apply(exp)

x.head(n=5)

c:\python35\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,AT,V,AP,RH
0,0.372521,0.291815,0.771591,0.638204
1,0.662040,0.669039,0.671863,0.449330
2,0.093484,0.249822,0.476862,0.892493
3,0.539660,0.568683,0.429349,0.684718
4,0.255241,0.216014,0.404355,0.952547


In [5]:
#splitting data into 2 set using sklearn train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=42)
print(xtrain.shape)
print(xtest.shape)

(6697, 4)
(2871, 4)


In [6]:
#We are going to use different model to predict PE and evaluate score of different models
#LinearRegression from sklearn.linear_models

reg1 = lm.LinearRegression()
reg1.fit(xtrain.values,ytrain.values)

predictions_test = reg1.predict(xtest.values)
predictions_train = reg1.predict(xtrain.values)

msetest = mse(ytest.values,predictions_test)
msetrain = mse(ytrain.values,predictions_train)

print("Mean squared Error of test data set : " + str(msetest))
print("Mean squared Error of train data set: " + str(msetrain))

Mean squared Error of test data set : 21.2398569382
Mean squared Error of train data set: 20.5808397257


In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation , Dropout
from keras.losses import mean_squared_error
import keras

Using TensorFlow backend.


In [8]:
#using keras we create 4 laeyer (input hidden hidden ouput) feedforward neural network... to predict PE

model = Sequential()
model.add(Dense(32,activation='linear',input_dim=4,init='uniform'))
model.add(Dense(32,activation='linear',init='uniform'))
model.add(Dense(1,activation='linear',init='uniform'))

model.compile(loss='mean_squared_error', 
              optimizer=keras.optimizers.Adam(lr=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0001),
              metrics=['mean_squared_error', 'acc'])

model.fit(xtrain.values,ytrain.values,batch_size=32,validation_data=(xtest.values,ytest.values),epochs=20,verbose=0)

pred_train = model.predict(xtrain.values)
pred_test = model.predict(xtest.values)

testscore = mse(ytest.values,pred_test)
trainscore = mse(ytrain.values,pred_train)

print("Mean squared Error of test data set : " + str(testscore))
print("Mean squared Error of train data set: " + str(trainscore))

c:\python35\lib\site-packages\ipykernel\__main__.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="linear", kernel_initializer="uniform", input_dim=4)`
c:\python35\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="linear", kernel_initializer="uniform")`
c:\python35\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="linear", kernel_initializer="uniform")`


Mean squared Error of test data set : 22.7531226369
Mean squared Error of train data set: 22.6733028249


In [49]:
b_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=6),
                          n_estimators=200,loss='square',random_state=42)
b_reg.fit(xtrain.values,ytrain.values)

predictions_test = b_reg.predict(xtest.values)
predictions_train = b_reg.predict(xtrain.values)

msetest = mse(ytest.values,predictions_test)
msetrain = mse(ytrain.values,predictions_train)

print("Mean squared Error of test data set : " + str(msetest))
print("Mean squared Error of train data set: " + str(msetrain))

c:\python35\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean squared Error of test data set : 16.7771577596
Mean squared Error of train data set: 14.1221087316
